# ReaDDy introduction

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import contextlib as cl
import time

import readdy._internal as api
api.set_logging_level("debug")

In [2]:
sim = api.Simulation()
sim.set_kernel("SingleCPU")
sim.box_size = api.Vec(60, 20, 20)
sim.periodic_boundary = [True, True, True]

#### Particle types for particles subject to brownian motion can be defined

In [3]:
sim.register_particle_type("A", diffusion_coefficient=1.0, radius=1.0)
sim.register_particle_type("B", diffusion_coefficient=1.0, radius=1.0)
sim.register_particle_type("C", diffusion_coefficient=0.1, radius=1.0)
sim.register_particle_type("Source A", diffusion_coefficient=0, radius=1.0)
sim.register_particle_type("Source B", diffusion_coefficient=0, radius=1.0)

4

#### which can interact with potentials

In [4]:
sim.register_potential_harmonic_repulsion("A", "A", force_constant=10)
sim.register_potential_harmonic_repulsion("B", "B", force_constant=10)
sim.register_potential_box("Source A", force_constant=10, origin=api.Vec(-25, -3, -3), 
                           extent=api.Vec(50, 6, 6), consider_particle_radius=False)
sim.register_potential_box("Source B", force_constant=10, origin=api.Vec(-25, -3, -3), 
                           extent=api.Vec(50, 6, 6), consider_particle_radius=False)
sim.register_potential_box("A", force_constant=10, origin=api.Vec(-25, -3, -3), 
                           extent=api.Vec(50, 6, 6), consider_particle_radius=False)
sim.register_potential_box("B", force_constant=10, origin=api.Vec(-25, -3, -3), 
                           extent=api.Vec(50, 6, 6), consider_particle_radius=False)

5

add particles

In [5]:
for idx, particle_type in enumerate(["Source A", "Source B"]):
    for i in range(20):
        yz = np.random.uniform(-2.5, 2.5, size=2)
        x = np.random.uniform(-25, -24) if idx == 0 else np.random.uniform(24, 25)
        sim.add_particle(particle_type, api.Vec(x, yz[0], yz[1]))

#### particles can also interact with reactions

- Conversion type reactions A -> B
- Fusion type reactions A+B -> C
- Fission type reactions A -> B+C
- Enzymatic type reactions A+C -> B+C

In [6]:
sim.register_reaction_fission("Factory creation A", from_type="Source A", to_type1="Source A", 
                              to_type2="A", rate=1.0, product_distance=1., weight1=.0, weight2=1.)
sim.register_reaction_fusion("Factory absorption A", from_type1="Source A", from_type2="A", 
                             to_type="Source A", rate=1., educt_distance=1., weight1=.0, weight2=1.)
sim.register_reaction_fission("Factory creation B", from_type="Source B", to_type1="Source B", 
                              to_type2="B", rate=1.0, product_distance=1., weight1=.0, weight2=1.)
sim.register_reaction_fusion("Factory absorption B", from_type1="Source B", from_type2="B", 
                             to_type="Source B", rate=1., educt_distance=1., weight1=.0, weight2=1.)
sim.register_reaction_fusion("Annihilation", from_type1="A", from_type2="B", to_type="C", rate=1., 
                             educt_distance=1.)
sim.register_reaction_decay("C Decay", particle_type="C", rate=1.)

2

### run the simulation

In [7]:
traj_handle = sim.register_observable_flat_trajectory(stride=1)
with cl.closing(api.File("out.h5", api.FileAction.CREATE, api.FileFlag.OVERWRITE)) as f:
    traj_handle.enable_write_to_file(f, "traj", 5000)
    t1 = time.perf_counter()
    sim.run_scheme_readdy(True)\
        .write_config_to_file(f)\
        .configure_and_run(10000, 1e-2)
    t2 = time.perf_counter()
print("Simulated", t2 - t1, "seconds")

Simulated 13.202293875000578 seconds


## output

In [8]:
t1 = time.perf_counter()
api.convert_xyz("./out.h5", "traj", "./traj.xyz", radii={"Source A": 1., "Source B": 1., "A": 1., "B": 1., "C": 1.})
t2 = time.perf_counter()
print("Conversion took", t2 - t1, "seconds")

Conversion took 8.064319133000026 seconds


## Observables